In [ ]:
import csv

def txt_to_csv(input_file, output_file):
    with open(input_file, 'r') as txt_file, open(output_file, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Index', 'Number', 'Name', 'Inning', 'Description', 'Balls', 'Strikes', 'Hits', 'Out # TOP', 'Out # BOTTOM'])

        start_conversion = False
        combined_line = ""
        index = 1
        inning = "TOP"

        for line in txt_file:
            line = line.strip()

            if "PLAY BY PLAY" in line:
                start_conversion = True
                continue

            if start_conversion:
                if line.startswith("#") or line.startswith("TOP") or line.startswith("BOTTOM") or line.startswith("SUMMARY"):
                    if combined_line:
                        number = combined_line.split()[0][1:] if combined_line.startswith("#") else ""
                        name = " ".join(combined_line.split()[1:3]) if len(combined_line.split()) >= 3 else ""
                        description = combined_line[len(number) + len(name) + 3:].strip()
                        balls = description.count("Ball")
                        strikes = min(description.count("Swinging Strike") + description.count("Called Strike") + description.count("Foul"), 2)
                        hits = 1 if any(keyword in description for keyword in ["single", "double", "triple", "home run"]) else ""
                        out_top = 1 if "putout" in description and inning == "TOP" else ""
                        out_bottom = 1 if "putout" in description and inning == "BOTTOM" else ""
                        if number:
                            writer.writerow([index, number, name, inning, description, balls, strikes, hits, out_top, out_bottom])
                            index += 1
                        combined_line = ""

                        if "out number 3" in description or "foroutnumber 3" in description:
                            if inning == "TOP":
                                inning = "BOTTOM"
                            else:
                                inning = "TOP"

                    if line.startswith("#"):
                        combined_line = line
                else:
                    if combined_line:
                        combined_line += line.strip()
                    else:
                        combined_line = line

        # Write the last at-bat after reaching the end of the file
        if combined_line:
            number = combined_line.split()[0][1:] if combined_line.startswith("#") else ""
            name = " ".join(combined_line.split()[1:3]) if len(combined_line.split()) >= 3 else ""
            description = combined_line[len(number) + len(name) + 3:].strip()
            balls = description.count("Ball")
            strikes = min(description.count("Swinging Strike") + description.count("Called Strike") + description.count("Foul"), 2)
            hits = 1 if any(keyword in description for keyword in ["single", "double", "triple", "home run"]) else ""
            out_top = 1 if "putout" in description and inning == "TOP" else ""
            out_bottom = 1 if "putout" in description and inning == "BOTTOM" else ""
            if number:
                writer.writerow([index, number, name, inning, description, balls, strikes, hits, out_top, out_bottom])

# Usage example:
txt_to_csv('game_play_by_play_report.txt', 'output.csv')


In [8]:
import csv
import os
from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        self.set_font("Arial", "B", 12)
        self.cell(0, 10, "Batting Average Report", 0, 1, "C")
        self.ln(10)

    def chapter_title(self, pitcher_name, date):
        self.set_font("Arial", "B", 12)
        self.cell(0, 10, f"Pitcher: {pitcher_name}", ln=True)
        self.set_font("Arial", "", 10)
        self.cell(0, 10, f"Date: {date}", ln=True)
        self.ln(10)

    def chapter_body(self, data):
        self.set_font("Arial", "B", 10)
        headers = ["Pitch Count", "Hits", "Total", "Batting Average"]
        cell_width = self.w / 4

        for header in headers:
            self.cell(cell_width, 10, header, 1, 0, "C")
        self.ln()

        self.set_font("Arial", "", 10)
        for pitch_count, pitch_data in data.items():
            hits = pitch_data["hits"]
            total = pitch_data["total"]
            batting_average = pitch_data["batting_average"]

            self.cell(cell_width, 10, pitch_count, 1, 0, "C")
            self.cell(cell_width, 10, str(hits), 1, 0, "C")
            self.cell(cell_width, 10, str(total), 1, 0, "C")
            self.cell(cell_width, 10, f"{batting_average:.3f}", 1, 0, "C")
            self.ln()

        self.ln(10)

def generate_pdf_file(date, pitcher_name, data, output_folder):
    pdf = PDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()

    pdf.chapter_title(pitcher_name, date)
    pdf.chapter_body(data)

    output_filename = os.path.join(output_folder, f"{pitcher_name}_{date}.pdf")
    pdf.output(output_filename)

def count_outs(input_file, output_file):
    pitchers_top = []
    pitchers_bottom = []
    current_pitcher_top = None
    current_pitcher_bottom = None

    with open(input_file, 'r') as csv_file:
        reader = csv.DictReader(csv_file)
        rows = list(reader)  # Read all rows into a list

        fieldnames = reader.fieldnames + ['Pitcher Top', 'Pitcher Bottom']

        while True:
            pitcher_name = input("Enter a pitcher's name for TOP (or 'done' to finish): ")
            if pitcher_name.lower() == 'done':
                break

            at_bats = int(input(f"Enter the number of batters faced for {pitcher_name} in TOP: "))
            pitchers_top.append({'name': pitcher_name, 'at_bats': at_bats, 'hits': {}, 'totals': {}})

        while True:
            pitcher_name = input("Enter a pitcher's name for BOTTOM (or 'done' to finish): ")
            if pitcher_name.lower() == 'done':
                break

            at_bats = int(input(f"Enter the number of batters faced for {pitcher_name} in BOTTOM: "))
            pitchers_bottom.append({'name': pitcher_name, 'at_bats': at_bats, 'hits': {}, 'totals': {}})

        at_bats_count_top = 0
        at_bats_count_bottom = 0
        pitcher_index_top = 0
        pitcher_index_bottom = 0
        current_pitcher_top = pitchers_top[pitcher_index_top % len(pitchers_top)]  # Set initial pitcher for TOP
        current_pitcher_bottom = pitchers_bottom[pitcher_index_bottom % len(pitchers_bottom)]  # Set initial pitcher for BOTTOM

        for row in rows:
            if row['Inning'] == 'TOP':
                if at_bats_count_top >= current_pitcher_top['at_bats']:
                    pitcher_index_top += 1
                    current_pitcher_top = pitchers_top[pitcher_index_top % len(pitchers_top)]
                    at_bats_count_top = 0  # Reset at-bats count for the new pitcher

                row['Pitcher Top'] = current_pitcher_top['name']
                pitch_count = f"{row['Balls']}B-{row['Strikes']}S"

                if pitch_count not in current_pitcher_top['totals']:
                    current_pitcher_top['totals'][pitch_count] = {'hits': 0, 'total': 0, 'batting_average': 0.0}
                    current_pitcher_top['totals'][pitch_count]['total'] += 1

                if int(row['Hits']) > 0:
                    current_pitcher_top['totals'][pitch_count]['hits'] += 1

                total = current_pitcher_top['totals'][pitch_count]['total']
                hits = current_pitcher_top['totals'][pitch_count]['hits']

                if total > 0:
                    batting_average = hits / total
                else:
                    batting_average = 0.0

                current_pitcher_top['totals'][pitch_count]['batting_average'] = batting_average

                at_bats_count_top += 1
            elif row['Inning'] == 'BOTTOM':
                if at_bats_count_bottom >= current_pitcher_bottom['at_bats']:
                    pitcher_index_bottom += 1
                    current_pitcher_bottom = pitchers_bottom[pitcher_index_bottom % len(pitchers_bottom)]
                    at_bats_count_bottom = 0  # Reset at-bats count for the new pitcher

                row['Pitcher Bottom'] = current_pitcher_bottom['name']
                pitch_count = f"{row['Balls']}B-{row['Strikes']}S"

                if pitch_count not in current_pitcher_bottom['totals']:
                    current_pitcher_bottom['totals'][pitch_count] = {'hits': 0, 'total': 0, 'batting_average': 0.0}

                current_pitcher_bottom['totals'][pitch_count]['total'] += 1

                if int(row['Hits']) > 0:
                    current_pitcher_bottom['totals'][pitch_count]['hits'] += 1

                total = current_pitcher_bottom['totals'][pitch_count]['total']
                hits = current_pitcher_bottom['totals'][pitch_count]['hits']

                if total > 0:
                    batting_average = hits / total
                else:
                    batting_average = 0.0

                current_pitcher_bottom['totals'][pitch_count]['batting_average'] = batting_average

                at_bats_count_bottom += 1

    # Write the modified data to a new CSV file
    with open(output_file, 'w', newline='') as csv_output:
        writer = csv.DictWriter(csv_output, fieldnames=fieldnames)
        writer.writeheader()

        for row in rows:
            writer.writerow(row)

    return pitchers_top, pitchers_bottom


def append_data_to_file(file_path, pitcher_data):
    fieldnames = ['Pitch Count', 'Hits', 'Total', 'Batting Average']

    file_exists = os.path.isfile(file_path)

    with open(file_path, 'a', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        if not file_exists:
            writer.writeheader()

        existing_data = {}
        if file_exists:
            with open(file_path, 'r') as existing_file:
                reader = csv.DictReader(existing_file)
                for row in reader:
                    pitch_count = row['Pitch Count']
                    existing_data[pitch_count] = {
                        'Hits': int(row['Hits']),
                        'Total': int(row['Total']),
                        'Batting Average': float(row['Batting Average'])
                    }

        for pitch_count, data in pitcher_data['totals'].items():
            hits = data['hits']
            total = data['total']
            batting_average = data['batting_average']

            if pitch_count in existing_data:
                existing_hits = existing_data[pitch_count]['Hits']
                existing_total = existing_data[pitch_count]['Total']

                hits += existing_hits
                total += existing_total

                if total > 0:
                    batting_average = hits / total
                else:
                    batting_average = 0.0

            writer.writerow({
                'Pitch Count': pitch_count,
                'Hits': hits,
                'Total': total,
                'Batting Average': batting_average
            })

input_date = input("Enter the date (YYYY-MM-DD): ")
input_folder = "Pitcher Data/CSV Game Files"
output_folder = "Pitcher Data/PDF Reports"

input_filename = f"{input_date}_output.csv"
input_file = os.path.join(input_folder, input_filename)
output_file = os.path.join(output_folder, input_filename)

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

pitchers_top, pitchers_bottom = count_outs(input_file, output_file)

print("Pitchers and their recorded at-bats:")
print("TOP:")
for pitcher in pitchers_top:
    print(f"{pitcher['name']}: {pitcher['at_bats']} at-bats")

print("\nBOTTOM:")
for pitcher in pitchers_bottom:
    print(f"{pitcher['name']}: {pitcher['at_bats']} at-bats")

print("\nPitchers and their batting average by pitch count:")
print("TOP:")
for pitcher in pitchers_top:
    print(f"{pitcher['name']}:")
    for pitch_count, data in pitcher['totals'].items():
        batting_average = data['batting_average']
        print(f"{pitch_count}: {batting_average:.3f} ({data['hits']} hits out of {data['total']} total)")
    print() 

for pitcher in pitchers_top:
    pitcher_name = pitcher['name']
    pitcher_data = pitcher['totals']
    generate_pdf_file(input_date, pitcher_name, pitcher_data, output_folder)

print("BOTTOM:")
for pitcher in pitchers_bottom:
    print(f"{pitcher['name']}:")
    for pitch_count, data in pitcher['totals'].items():
        batting_average = data['batting_average']
        print(f"{pitch_count}: {batting_average:.3f} ({data['hits']}")




KeyboardInterrupt: Interrupted by user

In [7]:
pip install FPDF

  Preparing metadata (setup.py) ... done
  Created wheel for FPDF: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=ac6c7cef7c4b823c785faf00c2f143e5e4a2da67cf50b3ce7a8c2232ee159eaa
  Stored in directory: /home/codespace/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built FPDF

[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
